In [1]:
# general imports
import os
import imageio
import mobie

In [2]:
# location of the input data. 
# the example data used in this notebook is available via this link:
input_data_path = "data"

# the location of the mobie project that will be created
# we recommend that the mobie project folders have the structure <PROECJT_ROOT_FOLDER/data>
# the folder 'data' will contain the sub-folders for individual datasets
mobie_project_folder = "mobie-project-01/data"

# name of the dataset that will be created.
# one project can contain multiple datasets
dataset_name = "LM-dataset"
dataset_folder = os.path.join(mobie_project_folder, dataset_name)

# the platform and number of jobs used for computation.
# choose 'local' to run computations on your machine.
# for large data, it is also possible to run computation on a cluster;
# for this purpose 'slurm' (for slurm cluster) and 'lsf' (for lsf cluster) are currently supported
target = "local"
max_jobs = 4

## Initialize the dataset

First, we need to initialize the dataset. This step includes generating the top-level project folder (if it's not present already), the subfolders for the new dataset and adding the "default" image for this dataset.
All these steps are performed by the function `add_image`.

This function accepts input image data in different formats. The input data is specified with the arguments
`input_path`, which specifies the file path and `input_key`, which specifies the internal path or search patterns.
- tif images (2d or 3d) - for this option set `input_key=''`
- folder with image files - for this option `input_key` needs to be the glob pattern for the image files, e.g `input_key='*.tif'` to load all tif files
- hdf5 file - `input_key` needs to be the internal file path
- n5 or zarr file - `input_key` needs to be the internal file path

The input files will be copied into the project folder in the [bdv.n5 dataformat](https://github.com/bigdataviewer/bigdataviewer-core/blob/master/BDV%20N5%20format.md) and an image pyramid will be created through consecutive downsampling.

To efficiently process large files the inputs should be in hdf5, n5 or zarr format.
Note that all inputs need to be either 2d or 3d images (volumes).
Multi-channel images (volumes) should be seperated into their channels and then each channel added individually (see `Adding image data` below).

In [3]:
input_data_path = "data"
# The 'default' image for our example dataset is a 2d EM slice showing an overview of the dataset.
input_file = os.path.join(input_data_path, "blobs.ome.tif")

# This is the name that will be given to the image source in mobie.
raw_name = "overview"
# The name of the menu from which the image can be added to the viewer.
# Here, we choose "em", because this is an EM image slice.
menu_name = "lm"

# We need some metadata to create the n5-file in big-data-viewer format:
# - unit: the phyiscal unit of the coordinate system
# - resolution: the size of one voxel in the physical unit, this needs to be a tuple/list of length 3,
#               specifying the size for each of the 3 spatial dimensions
# - chunks: the size of the chunks (in voxels) that are used to store the output file.
#           good choices are usually (1, 512, 512) for 2d data and (64, 64, 64) for 3d data
# - scale_factors: the scale factors used for downsampling the input when creating the image pyramid
#                  this needs to be a list, where each entry specifies the scale factors for the 3 axes.
# Note that axes are listed in the order ZYX for the resolution, chunks and scale factors
# (in the java implementation of mobie / big-data-viewer the axis convention is XYZ).
# Also note that the values for all three axes (ZYX) need to be specified. In the case of 2d data, the value
# for Z should be set to 1.
unit = "nanometer"
resolution = (1., 454, 454)
chunks = (1, 64, 64)
scale_factors = 4*[[1, 2, 2]]

mobie.add_image(
    input_path=input_file, 
    input_key='',  # the input is a single tif image, so we leave input_key blank
    root=mobie_project_folder,
    dataset_name=dataset_name,
    image_name=raw_name,
    menu_name=menu_name,
    resolution=resolution,
    chunks=chunks,
    scale_factors=scale_factors,
    is_default_dataset=True,  # mark this dataset as the default dataset that will be loaded by mobie
    target=target,
    max_jobs=max_jobs,
    unit=unit,
    file_format="ome.zarr"
)

c:\Users\xcamra\anaconda3\envs\mobie2-env\lib\site-packages\luigi\parameter.py:279: UserWarning: Parameter "dtype" with value "None" is not of type string.
  warnings.warn('Parameter "{}" with value "{}" is not of type string.'.format(param_name, param_value))
DEBUG: Checking if DownscalingWorkflow(tmp_folder=tmp_LM-dataset_overview, max_jobs=4, config_dir=tmp_LM-dataset_overview\configs, target=local, dependency=DummyTask, input_path=data\blobs.ome.tif, input_key=, scale_factors=[[1, 2, 2], [1, 2, 2], [1, 2, 2], [1, 2, 2]], halos=[[1, 2, 2], [1, 2, 2], [1, 2, 2], [1, 2, 2]], dtype=None, int_to_uint=False, metadata_format=ome.zarr, metadata_dict={"resolution": [1.0, 454, 454], "unit": "nanometer", "setup_name": "overview"}, output_path=mobie-project-01/data\LM-dataset\images\ome-zarr\overview.ome.zarr, output_key_prefix=, force_copy=False, skip_existing_levels=False, scale_offset=0) is complete
c:\Users\xcamra\anaconda3\envs\mobie2-env\lib\site-packages\luigi\parameter.py:279: UserWarn

RuntimeError: Downscaling failed

In [ ]:
meta = mobie.metadata.read_dataset_metadata(dataset_folder)

In [ ]:
meta["is2D"]

In [ ]:
meta["is2D"] = True
mobie.metadata.write_dataset_metadata(dataset_folder, meta)